# Sea ice forecasting using the IceNet library

## Context
### Purpose
This notebook demonstrates the use of the [IceNet library](https://pypi.org/project/icenet/) for sea-ice forecasting trained using climate reanalysis and observational data.

### Description
[IceNet](https://github.com/icenet-ai/icenet/) is a python library that provides the ability to download, process, train and predict from end to end. Users can interact with IceNet either via the python interface or via a set of command-line interfaces (CLI) which provide a high-level interface that covers the above abilities.

This notebook demonstrates the use of the python library api for forecasting sea ice for a reduced dataset to demonstrate its capabilities. The final output of interest are maps of sea ice concentration.

### Modelling approach
IceNet is a probabilistic, deep learning sea ice forecasting system. It utilises ensemble modelling of U-Net networks to generate daily forecasts of sea ice condition, trained on climate reanalysis and sea ice observational data (the built-in downloaders within IceNet are extensible). The original IceNet research model, published in Nature Communications ([Seasonal Arctic sea ice forecasting with probabilistic deep learning](https://www.nature.com/articles/s41467-021-25257-4)) was trained on climate simulations and observational data to forecast the next 6 months of monthly-averaged sea ice concentration maps, while the updated library utilises daily inputs and is able to generate daily forecast outputs for a variable forecast time period. The python library showcased here is a heavily refactored version of the original research code from the original publication that has been developed for operational forecasting. The core UNet architecture is implemented using Tensorflow, however, the library architecture allows other backend libraries to be utilised and wrapped around the IceNet library ecosystem.

### Highlights
 * [1. Setup](#Setup) the environment and project structure.
 * [2. Download](#Download) sea ice concentration data as training data.
 * [3. Process](#Process) downloaded data, and generate cached datasets to speed up training.
 * [4. Train](#Train) the neural network and generate checkpoint and model output.
 * [5. Predict](#Predict) for defined dates.
 * [6. Visualisation](#Visualisation) of the prediction output.

### Contributions

#### Notebook
* James Byrne (author), British Antarctic Survey, [@JimCircadian](https://github.com/JimCircadian)
* Bryn Noel Ubald (author), British Antarctic Survey, [@bnubald](https://github.com/bnubald)

#### Modelling codebase
* James Byrne (Code author)
* Tom Andersson (Science author)
* Bryn Noel Ubald (Code maintainer and contributor)

:::{note}
__More in-depth notebooks on using IceNet are available [in this repository](https://github.com/icenet-ai/icenet-notebooks)__, including the use of ensemble modelling, and library extension to use different backends and neural network models.

Contact me at _jambyr \<at\> bas.ac.uk_ for anything else...
:::


#### Modelling publications
Andersson, T.R., Hosking, J.S., Pérez-Ortiz, M. et al. Seasonal Arctic sea ice forecasting with probabilistic deep learning. Nat Commun 12, 5124 (2021). https://doi.org/10.1038/s41467-021-25257-4

:::{note}
The original paper and notebook used a combination of climate simulations and observational data to forecast the next 6 months of monthly-averaged sea ice concentration. Since then, the original code has been refactored into a new `icenet` library as showcased in this notebook.

This library supports sea ice forecasting on a daily resolution rather than monthly-averaged. It has been developed significantly since the original paper to operationalise the code, and to that end, there are multiple ways of interacting with the library to help enable development of sea ice forecasting and facilitate model development. More of these interfaces and use-case scenarios are covered in the [icenet-notebooks](https://github.com/icenet-ai/icenet-notebooks) repository.
:::

#### Source code

There are multiple relevant code bases depending on the usage scenario, but the main python IceNet library is located [here](https://github.com/icenet-ai/icenet). If of interest, other related repositories can be found in the [icenet-ai](https://github.com/icenet-ai/) github organisation.

#### Involved organisations
The Alan Turing Institute and British Antarctic Survey

___
# Setup

## Load libraries
Load some of the common libraries required.

In [1]:
import os
import random
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd
import tensorflow as tf

# We also set the logging level so that we get some feedback from the API
import logging
logging.basicConfig(level=logging.INFO)

# Set up logging level
import dask
import distributed
import xarray
logging.getLogger('icenet').setLevel(logging.INFO)
logging.getLogger('dask').setLevel(logging.ERROR)
logging.getLogger('distributed').setLevel(logging.ERROR)

The following imports modules from the IceNet library as preparation for the downloaders. Whose instantiation describes the interactions with the upstream APIs/data interfaces used to source various types of data. 

In [2]:
from icenet.data.sic.mask import Masks
from icenet.data.interfaces.cds import ERA5Downloader
from icenet.data.sic.osisaf import SICDownloader

## Set project structure

In [3]:
notebook_folder = './notebook'
if not os.path.exists(notebook_folder):
    os.makedirs(notebook_folder)

___
# Download

In this section, we download all required data with our extended date range. All downloaders inherit a `download` method from the `Downloader` class in [`icenet.data.producers`](https://github.com/icenet-ai/icenet/blob/main/icenet/data/producers.py), which also contains two other data producing classes `Generator` (which Masks inherits from) and `Processor` (used in the next section), each providing abstract implementations that multiple classes derive from.

### Masked regions

We start here with generating the masks for training/prediction. This includes regions where sea ice does not form, land regions, and the [polar hole](https://blogs.egu.eu/divisions/cr/2016/10/14/image-of-the-week-the-polar-hole/).

:::note
This data downloaded from OSI-SAF over an FTP server which is blocked and not accessible via Binder, hence, the final datafile that the next cell would output for these range of dates is included in the repository.

It does not change the workflow or the code, the class will simply skip over the date ranges that have already been downloaded.
:::

In [4]:
masks = Masks(north=False, south=True)
masks.generate(save_polarhole_masks=False)

INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200001021200.nc already exists
INFO:root:Saving ./data/masks/south/masks/active_grid_cell_mask_01.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200002021200.nc already exists
INFO:root:Saving ./data/masks/south/masks/active_grid_cell_mask_02.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200003021200.nc already exists
INFO:root:Saving ./data/masks/south/masks/active_grid_cell_mask_03.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200004021200.nc already exists
INFO:root:Saving ./data/masks/south/masks/active_grid_cell_mask_04.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200005021200.nc already exists
INFO:root:Saving ./data/masks/south/masks/active_grid_cell_mask_05.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200006021200.nc already exists
INFO:root:Saving ./data/masks/south/masks/active_grid_cell_mask_06.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200007021200.nc already exists
INFO:root:Savi

## Climate and Ocean data

Climate and ocean data are obtained from the [Climate Data Store (CDS)](https://cds.climate.copernicus.eu/).

* The climate data used for training is from [ERA5](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview) reanalysis which covers the global climate since 1940 to the present time.

* The Ocean data used is from [ORA5](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-oras5?tab=overview) which also uses a reanalysis approach and contains global ocean and sea-ice reanalysis data.

Since these are both obtained from reanalysis, they are a combination of physical models and observational data. Due to the reanalysis approach, there is no temporal or spatial gap in the downloaded data. Both of these sets of data are obtained from the ECMWF's (European Centre for Medium-Range Weather Forecast) reanalysis systems.

Please see the above links for more details on these datasets.

The downloader implementation of this data in IceNet utilises the CDS API which requires registration and configuration of an API key before downloading. The registration is free, please see [the official CDS API how-to](https://cds.climate.copernicus.eu/api-how-to) for more instructions on how to set this up.

:::{note}
Since the ERA5 and ORA5 data downloads require registration before download, this demonstrator will only download and use observed sea ice concentration data for training.

Once the key is configured correctly, you can utilise the `ERA5Downloader` class, for example, to download climate variables and use it for training the model in addition to the sea ice concentration data that is downloaded further below.
:::


```python
era5 = ERA5Downloader(
    var_names=["tas", "zg", "uas", "vas"],      # Name of variables to download
    dates=[                                     # Dates to download the variable data for
        pd.to_datetime(date).date()
        for date in pd.date_range("2020-01-01", "2020-04-31", freq="D")
    ],
    path=data_dir,                              # Location to download data to (default is `./data`)
    delete_tempfiles=True,                      # Whether to delete temporary downloaded files
    levels=[None, [250, 500], None, None],      # The levels at which to obtain the variables for (e.g. for zg, it is the pressure levels)
    max_threads=4,                              # Maximum number of concurrent downloads
    north=False,                                # Boolean: Whether require data across northern hemisphere
    south=True,                                 # Boolean: Whether require data across southern hemisphere
    use_toolbox=False)                          # Experimental, alternative download method
era5.download()                                 # Start downloading
```

The `ERA5Downloader` inherits from `ClimateDownloader`, from which several implementations derive their functionality. Two particularly useful methods shown below allow the downloaded data to be converted to the same grid and orientation as the OSISAF sea-ice concentration (SIC) data in the next cell.

```python
era5.regrid()                                   # Map data onto common EASE2 grid
era5.rotate_wind_data()                         # Rotate wind data to correct orientation
```

## Sea-ice concentration (SIC) data

The sea-ice concentration data use for training is obtained from [OSI SAF](https://osi-saf.eumetsat.int/products/sea-ice-products).

The SIC is defined as the fraction of a grid cell that is covered in sea-ice.

You will notice a familiar interface as with the `ERA5Downloader` class with the `SICDownloader` class, with similar input arguments.

:::note
In a similar manner to the Masks data download above, the OSI-SAF data is downloaded over an FTP server which is blocked and not accessible via Binder, hence, the final datafile that the next cell would output for these range of dates is included in the repository.

It does not change the workflow or the code, the class will simply skip over the date ranges that have already been downloaded.
:::

In [5]:
sic = SICDownloader(
    dates=[
        pd.to_datetime(date).date()             # Dates to download the variable data for
        for date in pd.date_range("2020-01-01", "2020-04-30", freq="D")
    ],
    delete_tempfiles=True, # Whether to delete temporary downloaded files
    north=False,           # Boolean: Whether to use mask for this region
    south=True,            # Boolean: Whether to use mask for this region
    parallel_opens=True,   # Boolean: Whether to use `dask.delayed` to open and preprocess multiple files in parallel
)

sic.download()

INFO:root:Downloading SIC datafiles to .temp intermediates...
INFO:root:Excluding 121 dates already existing from 121 dates requested.
INFO:root:Opening for interpolation: ['./data/osisaf/south/siconca/2020.nc']
INFO:root:Processing 0 missing dates


___
# Process

Similarly to the downloaders, each data producer (be it a `Downloader` or `Generator`) has a respective `Processor` that converts the `./notebook/data/` products into a normalised, preprocessed dataset under `./notebook/processed/`.

Firstly, to make life a bit easier, we set up some variables. In this case we're creating a train/validate/test split out of the 2020 data in a fairly naive manner.

In [6]:
processing_dates = dict(
    train=[pd.to_datetime(el) for el in pd.date_range("2020-01-01", "2020-02-29")],
    val=[pd.to_datetime(el) for el in pd.date_range("2020-04-03", "2020-04-23")],
    test=[pd.to_datetime(el) for el in pd.date_range("2020-04-01", "2020-04-02")],
)
processed_name = "notebook_api_data"

Next, we create the data producer and configure them for the dataset we want to create.

These modules import the Processing modules for the downloaded data.

In [7]:
from icenet.data.processors.era5 import IceNetERA5PreProcessor # Unused in this demonstrator notebook
from icenet.data.processors.meta import IceNetMetaPreProcessor
from icenet.data.processors.osi import IceNetOSIPreProcessor

In [8]:
osi = IceNetOSIPreProcessor(
    ["siconca"],                # Absolute normalised variables
    [],                         # Variables defined as deviations from an aggregated norm
    processed_name,
    processing_dates["train"],
    processing_dates["val"],
    processing_dates["test"],
    linear_trends=tuple(),
    north=False,
    south=True,
)

meta = IceNetMetaPreProcessor(
    processed_name,
    north=False,
    south=True,
)

As mentioned above, this demonstrator does not use the ERA5 climate reanalysis data since an private API key should be set up per user to access the CDS API ([how-to here](https://cds.climate.copernicus.eu/api-how-to)). However, if it was set up, the ERA5 data can also be preprocessed by:

```python
pp = IceNetERA5PreProcessor(
    ["uas", "vas"],             # Absolute normalised variables
    ["tas", "zg500", "zg250"],  # Variables defined as deviations from an aggregated norm
    processed_name,
    processing_dates["train"],
    processing_dates["val"],
    processing_dates["test"],
    linear_trends=tuple(),
    north=False,
    south=True
)
```

Next, we initialise the data processors using `init_source_data` which scans the data source directories to understand what data is available for processing based on the parameters. Since we named the processed data `"notebook_api_data"` above, it will create a data loader config file, `loader.notebook_api_data.json`, in the current directory.

In [9]:
osi.init_source_data(
    lag_days=1,
)
osi.process()

meta.process()

INFO:root:Processing 60 dates for train category
INFO:root:Including lag of 1 days
INFO:root:Including lead of 93 days
INFO:root:No data found for 2019-12-31, outside data boundary perhaps?
INFO:root:Processing 21 dates for val category
INFO:root:Including lag of 1 days
INFO:root:Including lead of 93 days
INFO:root:Processing 2 dates for test category
INFO:root:Including lag of 1 days
INFO:root:Including lead of 93 days
INFO:root:Got 1 files for siconca
INFO:root:Opening files for siconca
INFO:root:Filtered to 121 units long based on configuration requirements
INFO:root:No normalisation for siconca
INFO:root:Loading configuration ./loader.notebook_api_data.json
INFO:root:Writing configuration to ./loader.notebook_api_data.json
INFO:root:Loading configuration ./loader.notebook_api_data.json
INFO:root:Writing configuration to ./loader.notebook_api_data.json


At this point the preprocessed data is ready to convert or create a configuration for the network dataset.

### Dataset creation

Now, we can create a dataset configuration for training the network. This can include cached data for the network in the format of a TFRecordDataset compatible set of tfrecords. To achieve this we create the `IceNetDataLoader`, which can both generate `IceNetDataSet` configurations (which easily provide the necessary functionality for training and prediction) as well as individual data samples for direct usage.

In [10]:
from icenet.data.loaders import IceNetDataLoaderFactory

implementation = "dask"
loader_config = f"loader.{processed_name}.json"
dataset_name = "api_dataset"
lag = 1

dl = IceNetDataLoaderFactory().create_data_loader(
    implementation,
    loader_config,
    dataset_name,
    lag,
    n_forecast_days=7,
    north=False,
    south=True,
    output_batch_size=1,
    generate_workers=1
)

INFO:root:Creating path: ./network_datasets/api_dataset
INFO:root:Loading configuration loader.notebook_api_data.json


We can see the loader config contains information about the data sources included and also the different dates to use for the training, validation and test sets:

In [11]:
dl._config

{'sources': {'osisaf': {'name': 'notebook_api_data',
   'implementation': 'IceNetOSIPreProcessor',
   'anom': [],
   'abs': ['siconca'],
   'dates': {'train': ['2020_01_01',
     '2020_01_02',
     '2020_01_03',
     '2020_01_04',
     '2020_01_05',
     '2020_01_06',
     '2020_01_07',
     '2020_01_08',
     '2020_01_09',
     '2020_01_10',
     '2020_01_11',
     '2020_01_12',
     '2020_01_13',
     '2020_01_14',
     '2020_01_15',
     '2020_01_16',
     '2020_01_17',
     '2020_01_18',
     '2020_01_19',
     '2020_01_20',
     '2020_01_21',
     '2020_01_22',
     '2020_01_23',
     '2020_01_24',
     '2020_01_25',
     '2020_01_26',
     '2020_01_27',
     '2020_01_28',
     '2020_01_29',
     '2020_01_30',
     '2020_01_31',
     '2020_02_01',
     '2020_02_02',
     '2020_02_03',
     '2020_02_04',
     '2020_02_05',
     '2020_02_06',
     '2020_02_07',
     '2020_02_08',
     '2020_02_09',
     '2020_02_10',
     '2020_02_11',
     '2020_02_12',
     '2020_02_13',
     '202

At this point we can either use `generate` or `write_dataset_config_only` to produce a ready-to-go `IceNetDataSet` configuration. Both of these will generate a dataset config, `dataset_config.api_dataset.json` (recall we set the dataset name as `api_dataset` above).

In [12]:
%%time
dl.generate()

INFO:root:Dashboard at localhost:8888
INFO:root:Using dask client <Client: 'tcp://127.0.0.1:37780' processes=1 threads=1, memory=503.20 GiB>
INFO:root:60 train dates to process, generating cache data.
/data/hpcdata/users/bryald/miniconda3/envs/icenet-edsbookd/lib/python3.11/site-packages/xarray/core/dataset.py:271: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
/data/hpcdata/users/bryald/miniconda3/envs/icenet-edsbookd/lib/python3.11/site-packages/xarray/core/dataset.py:271: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(
INFO:root:Finished output ./network_datasets/api_dataset/south/train/00000000.tfrecord
INFO:root:Finished output ./network_datasets/api_dataset/south/train/00000001.tfreco

CPU times: user 19.9 s, sys: 2.35 s, total: 22.3 s
Wall time: 1min 49s


To generate samples from this dataset, we can use the `.generate_sample()` method, which returns the inputs `x`, `y` and sample weights `sw`:

In [13]:
x, y, sw = dl.generate_sample(pd.Timestamp("2020-04-01"))

In [14]:
print(f"type(x): {type(x)}, x.shape: {x.shape}")
print(f"type(y): {type(y)}, y.shape: {y.shape}")
print(f"type(sw): {type(sw)}, sw.shape: {sw.shape}")

type(x): <class 'numpy.ndarray'>, x.shape: (432, 432, 4)
type(y): <class 'numpy.ndarray'>, y.shape: (432, 432, 7, 1)
type(sw): <class 'numpy.ndarray'>, sw.shape: (432, 432, 7, 1)


___
# Train

For single runs we can programmatically call `train_model` which defines the training process from start to finish. If we wanted to run mutliple models with different initialisation states (called ensemble modelling) to obtain some level of model uncertainty, it is recommended to use the [`model-ensembler`](https://github.com/JimCircadian/model-ensembler) tool which works outside of the API and is capable of controlling multiple CLI submissions. Customising an ensemble can be achieved through looking at the configuration in [the pipeline repository](https://github.com/antarctica/IceNet-Pipeline). That said, if workflow system integration (e.g. Airflow) is desired, integrating via this method is the way to go.

We can load the dataset configuration by creating an `IceNetDataSet` from the `dataset_config.api_dataset.json` file and using the `_config` method.

In [15]:
from icenet.data.dataset import IceNetDataSet

dataset_config = f"dataset_config.{dataset_name}.json"
dataset = IceNetDataSet(dataset_config, batch_size=1)
strategy = tf.distribute.get_strategy()

INFO:root:Loading configuration dataset_config.api_dataset.json
INFO:root:Training dataset path: ./network_datasets/api_dataset/south/train
INFO:root:Validation dataset path: ./network_datasets/api_dataset/south/val
INFO:root:Test dataset path: ./network_datasets/api_dataset/south/test


In [16]:
dataset._config

{'identifier': 'api_dataset',
 'implementation': 'DaskMultiWorkerLoader',
 'channels': ['siconca_abs_1', 'cos_1', 'land_1', 'sin_1'],
 'counts': {'train': 60, 'val': 21, 'test': 2},
 'dtype': 'float32',
 'loader_config': '/data/hpcdata/users/bryald/git/turing/icenet-edsbook/loader.notebook_api_data.json',
 'missing_dates': [],
 'n_forecast_days': 7,
 'north': False,
 'num_channels': 4,
 'shape': [432, 432],
 'south': True,
 'dataset_path': './network_datasets/api_dataset',
 'generate_workers': 1,
 'loss_weight_days': True,
 'output_batch_size': 1,
 'var_lag': 1,
 'var_lag_override': {}}

You can obtain the data loader that was used to create the dataset config via the `.get_data_loader()` method:

In [17]:
dataset.get_data_loader()

INFO:root:Loading configuration /data/hpcdata/users/bryald/git/turing/icenet-edsbook/loader.notebook_api_data.json


We can use `train_model` function to train the UNet model based on the above downloaded and processed data.

In [18]:
%%time
from icenet.model.train import train_model

run_name = "api_test_run"
seed = 42

trained_path, history = train_model(
    run_name=run_name,
    dataset=dataset,
    epochs=10,
    n_filters_factor=0.3,
    seed=seed,
    strategy=strategy,
    training_verbosity=2,
    workers=1
)

INFO:root:Creating network folder: ./results/networks/api_test_run
INFO:root:Adding tensorboard callback


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 432, 432, 4)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 432, 432, 19)         703       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 432, 432, 19)         3268      ['conv2d[0][0]']              
                                                                                                  
 batch_normalization (Batch  (None, 432, 432, 19)         76        ['conv2d_1[0][0]']            
 Normalization)                                                                               

INFO:root:Datasets: 60 train, 21 val and 2 test filenames
INFO:root:Reducing datasets to 1.0 of total files
INFO:root:Reduced: 60 train, 21 val and 2 test filenames
INFO:root:
Setting learning rate to: 9.999999747378752e-05



Epoch 1/10

Epoch 1: val_rmse improved from inf to 36.74448, saving model to ./results/networks/api_test_run/api_test_run.network_api_dataset.42.h5
60/60 - 39s - loss: 169.1611 - binacc: 52.9915 - mae: 24.0722 - rmse: 30.5039 - mse: 1734.8082 - val_loss: 245.4568 - val_binacc: 42.6351 - val_mae: 34.0166 - val_rmse: 36.7445 - val_mse: 1766.8466 - lr: 1.0000e-04 - 39s/epoch - 656ms/step


INFO:root:
Setting learning rate to: 9.999999747378752e-05



...

Epoch 10/10

Epoch 10: val_rmse improved from 18.92558 to 18.71253, saving model to ./results/networks/api_test_run/api_test_run.network_api_dataset.42.h5
60/60 - 31s - loss: 5.0998 - binacc: 97.7079 - mae: 2.1330 - rmse: 5.2964 - mse: 1781.5924 - val_loss: 63.6584 - val_binacc: 82.2293 - val_mae: 13.6606 - val_rmse: 18.7125 - val_mse: 2019.7556 - lr: 1.0000e-04 - 31s/epoch - 523ms/step


INFO:root:Saving network to: ./results/networks/api_test_run/api_test_run.network_api_dataset.42.h5


INFO:tensorflow:Assets written to: ./results/networks/api_test_run/api_test_run.model_api_dataset.42/assets


INFO:tensorflow:Assets written to: ./results/networks/api_test_run/api_test_run.model_api_dataset.42/assets


CPU times: user 1h 54s, sys: 6min 9s, total: 1h 7min 3s
Wall time: 5min 19s


As can be seen the training workflow is very standard for deep learning networks, with `train_model` wrapping up the training process with a lot of customisation of extraneous functionality.

___
# Predict

In much the same manner as with `train_model`, the `predict_forecast` method acts as a convenient entry point workflow system integration, CLI entry as well as an overridable method upon which to base custom implementations. Using the method directly relies on loading from a prepared (but perhaps not cached) dataset.

Some parameters are fed to `predict_forecast` that ideally shouldn't need to be specified (like `seed` and `n_filters_factor`) and might seem contextually odd. They're used to locate the appropriate saved network. *This will be cleaned up in a future version*.

In [19]:
%%time
from icenet.model.predict import predict_forecast

# Follows the naming convention used by the CLI version
output_dir = os.path.join(".", "results", "predict",
                          "custom_run_forecast",
                          "{}.{}".format(run_name, "42"))

predict_forecast(
    dataset_config=dataset_config,
    network_name=run_name,
    n_filters_factor=0.3,
    output_folder=output_dir,
    seed=seed,
    start_dates=[pd.to_datetime(el).date()
                 for el in pd.date_range("2020-04-01", "2020-04-02")],
    test_set=True,
)

INFO:root:Loading configuration dataset_config.api_dataset.json
INFO:root:Training dataset path: ./network_datasets/api_dataset/south/train
INFO:root:Validation dataset path: ./network_datasets/api_dataset/south/val
INFO:root:Test dataset path: ./network_datasets/api_dataset/south/test
INFO:root:Loading configuration /data/hpcdata/users/bryald/git/turing/icenet-edsbook/loader.notebook_api_data.json
INFO:root:Loading model from ./results/networks/api_test_run/api_test_run.network_api_dataset.42.h5...
INFO:root:Datasets: 120 train, 42 val and 4 test filenames
INFO:root:Processing test batch 1, item 0 (date 2020-04-01)
INFO:root:Running prediction 2020-04-01
INFO:root:Saving 2020-04-01 - forecast output (1, 432, 432, 7)
INFO:root:Processing test batch 1, item 1 (date 2020-04-02)
INFO:root:Running prediction 2020-04-02
INFO:root:Saving 2020-04-02 - forecast output (1, 432, 432, 7)


CPU times: user 6.22 s, sys: 730 ms, total: 6.95 s
Wall time: 1.71 s


The individual outputs of the above command are deposited into the following directory:

```bash
results/predict/custom_run_forecast/api_test_run.42/
├── 2020_04_01.npy
└── 2020_04_02.npy
```

The persistence and respective use of these results is then up to the user. They consist of numpy files for each test date that contain the following:
1. *forecast*: The neural network predicted forecast sea ice concentration data.
1. *outputs*: Outputs from the data loader used for training.
1. *sample_weights*: The sample weights used to weight training samples.

To generate __a CF-compliant NetCDF containing the forecasts requested__ we need to run `icenet_output`, these can then be post-processed. As an input to this command, we need to provide it with a `csv` containing the test dates. In this example, we use `printf` to generate the required file.

In [20]:
!printf "2020-04-01\n2020-04-02" | tee predict_dates.csv

2020-04-01
2020-04-02

In [21]:
!icenet_output -m -o ./results/predict custom_run_forecast api_dataset predict_dates.csv

[15-03-24 20:28:11 :INFO    ] - Loading configuration ./dataset_config.api_dataset.json
[15-03-24 20:28:11 :INFO    ] - Training dataset path: ./network_datasets/api_dataset/south/train
[15-03-24 20:28:11 :INFO    ] - Validation dataset path: ./network_datasets/api_dataset/south/val
[15-03-24 20:28:11 :INFO    ] - Test dataset path: ./network_datasets/api_dataset/south/test
/data/hpcdata/users/bryald/miniconda3/envs/icenet-edsbookd/lib/python3.11/site-packages/iris/__init__.py:354: FutureWarning: Ignoring a datum in netCDF load for consistency with existing behaviour. In a future version of Iris, this datum will be applied. To apply the datum when loading, use the iris.FUTURE.datum_support flag.
  cubes = _load_collection(uris, constraints, callback).cubes()
[15-03-24 20:28:12 :INFO    ] - Post-processing 2020-04-01
[15-03-24 20:28:12 :INFO    ] - Post-processing 2020-04-02
[15-03-24 20:28:12 :INFO    ] - Dataset arr shape: (2, 432, 432, 7, 2)
[15-03-24 20:28:12 :INFO    ] - Applying a

___
# Visualisation

Now that we have a prediction, we can visualise the binary sea ice concentration using some of the built-in tools in IceNet that utilise `cartopy` and `matplotlib`.

(Note: There are also some scripts in the [icenet-pipeline](https://github.com/icenet-ai/icenet-pipeline) repository that enable plotting common results such as `produce_op_assets.sh`)

Here, we are loading the prediction netCDF file we've just created in the previous step.

We are also using the `Masks` class from IceNet to create a land mask region that will mask out the land regions in the forecast plot.

In [22]:
from icenet.plotting.video import xarray_to_video as xvid
from icenet.data.sic.mask import Masks
from IPython.display import HTML
import xarray as xr, pandas as pd, datetime as dt

# Load our output prediction file
ds = xr.open_dataset("results/predict/custom_run_forecast.nc")
land_mask = Masks(south=True, north=False).get_land_mask()
ds.info()

xarray.Dataset {
dimensions:
	time = 2 ;
	yc = 432 ;
	xc = 432 ;
	leadtime = 7 ;

variables:
	int32 Lambert_Azimuthal_Grid() ;
		Lambert_Azimuthal_Grid:grid_mapping_name = lambert_azimuthal_equal_area ;
		Lambert_Azimuthal_Grid:longitude_of_projection_origin = 0.0 ;
		Lambert_Azimuthal_Grid:latitude_of_projection_origin = -90.0 ;
		Lambert_Azimuthal_Grid:false_easting = 0.0 ;
		Lambert_Azimuthal_Grid:false_northing = 0.0 ;
		Lambert_Azimuthal_Grid:semi_major_axis = 6378137.0 ;
		Lambert_Azimuthal_Grid:inverse_flattening = 298.257223563 ;
		Lambert_Azimuthal_Grid:proj4_string = +proj=laea +lon_0=0 +datum=WGS84 +ellps=WGS84 +lat_0=-90.0 ;
	float32 sic_mean(time, yc, xc, leadtime) ;
		sic_mean:long_name = mean sea ice area fraction across ensemble runs of icenet model ;
		sic_mean:standard_name = sea_ice_area_fraction ;
		sic_mean:short_name = sic ;
		sic_mean:valid_min = 0 ;
		sic_mean:valid_max = 1 ;
		sic_mean:ancillary_variables = sic_stddev ;
		sic_mean:grid_mapping = Lambert_Azimuth

We obtain the start date of the forecast we would like to plot

In [23]:
# Get the forecast start date
forecast_date = ds.time.values[0]
print(forecast_date)

2020-04-01T00:00:00.000000000


Now, we can use the built in plotting tool to visualise our forecast.

:::note
Since this is a demonstrator notebook, we have not trained our network for a prolonged period of time or for a large date range, but the plot below shows indicative results binary sea ice concentration.
:::

In [24]:
fc = ds.sic_mean.isel(time=0).drop_vars("time").rename(dict(leadtime="time"))
fc['time'] = [pd.to_datetime(forecast_date) \
              + dt.timedelta(days=int(e)) for e in fc.time.values]

anim = xvid(fc, 15, figsize=4, mask=land_mask)
HTML(anim.to_jshtml())

INFO:root:Inspecting data
INFO:root:Initialising plot
INFO:root:Animating
INFO:root:Not saving plot, will return animation
INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


## Summary

This notebook has demonstrated the use of the IceNet library to programmatically generate sea-ice forecasts via the `IceNet` python interface, across all stages of a machine learning workflow, from source data download to forecast visualisation. It demonstrated this by:

* Showing usage of the IceNet library to download different climate data variables ([ERA5](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview)) and sea ice concentration maps ([OSI-SAF](https://osi-saf.eumetsat.int/)) from different data sources via a similar api interface. (The ERA5 code was shown, but not run due to need for a free personal API key having been setup by the user)
* Processing of downloaded data to include normalisation and data caching to speed up the training.
* Showing the use of a high-level interface to the IceNet UNet model to train on different data sources and generate predictions of binary sea ice concentration.
* Visualising the predictions using the plotting tools within the IceNet library.
* If researching, consider [extending the functionality of the API to include revised or completely new implementations, such as additional data sources](https://github.com/icenet-ai/icenet-notebooks/blob/develop/05.library_extension.ipynb)

## Additional information
**Codebase**: [IceNet v0.2.7](https://pypi.org/project/icenet/0.2.7/).

**License**: The code in this notebook is licensed under the MIT License. The Environmental Data Science book is licensed under the Creative Commons by Attribution 4.0 license. See further details [here](https://github.com/alan-turing-institute/environmental-ds-book/blob/master/LICENSE.md).

**Contact**: If you have any suggestion or report an issue with this notebook, feel free to [create an issue](https://github.com/alan-turing-institute/environmental-ds-book/issues/new/choose) or send a direct message to [environmental.ds.book@gmail.com](mailto:environmental.ds.book@gmail.com).

### Extended usage

This has been a very simplified demonstrator notebook that covers the usage of IceNet only via the Python API. There are also command line interface and bash scripts that allow operationalising IceNet and simplify the typical workflows one might come across when generating sea ice forecasts.

To learn more about such approaches, there are a series of introductory notebooks (five notebooks at present) that cover both the fundamental and advanced usage scenarios of the IceNet library under the [icenet-notebooks](https://github.com/icenet-ai/icenet-notebooks) repository. They are as follows:

* [Introductory usage via CLI](https://github.com/icenet-ai/icenet-notebooks/blob/develop/01.cli_demonstration.ipynb): Demonstrates the usage of the different stages of icenet (download, process, train, predict and output) via a very high level command line interface.
* [Pipeline usage](https://github.com/icenet-ai/icenet-notebooks/blob/develop/02.cli_pipeline_demo.ipynb): Creating operational and reproducible end-to-end runs.
* [Data structure and analysis](https://github.com/icenet-ai/icenet-notebooks/blob/develop/03.data_and_forecasts.ipynb): Understand the structure of the data stores and products created by these workflows and what tools currently exist in IceNet to look over them.
* [Library usage](https://github.com/icenet-ai/icenet-notebooks/blob/develop/04.library_usage.ipynb): Understand how to programmatically perform an end to end run.
* [Library extension](https://github.com/icenet-ai/icenet-notebooks/blob/develop/05.library_extension.ipynb): Understand why and how to extend the IceNet library.

In [25]:
import icenet
icenet_version = icenet.__version__
print(f'IceNet version: {icenet_version}')

IceNet version: 0.2.7


In [26]:
from datetime import date
print(f'Last tested: {date.today()}')

Last tested: 2024-03-15
